In [2]:
import sys
sys.path.append("/root/HSE_diploma/")
sys.path.append("/root/HSE_diploma/evaluator/")

In [3]:
sys.path

['/root/HSE_diploma/ipynb',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 '/opt/conda/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg',
 '/root/HSE_diploma/',
 '/root/HSE_diploma/evaluator/']

In [4]:
import os
import json

import transformers
from peft import PeftModel
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model 
from datasets import load_dataset
from EvaluateTestSet import EvaluateTestSet
from transformers.integrations import TensorBoardCallback
from transformers import GenerationConfig



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


/root/HSE_diploma/prompter/templates/


In [5]:

def init_lora_model_and_tokenizer(default_model,
                             LORA_R,
                             LORA_ALPHA,
                             LORA_DROPOUT
                            ):


    """
        
    """
    model = LLaMAForCausalLM.from_pretrained(
    default_model,
    load_in_8bit=True,
    device_map="auto",
    )
    tokenizer = LLaMATokenizer.from_pretrained(
        default_model, add_eos_token=True
    )

    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, config)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token

    return model, tokenizer

In [8]:
CONFIG_PATH = "/root/HSE_diploma/experiments_configs/"
EXPERIMENTS_PATH = "/root/experiments/"
experiment_name = "t2c_concode_220428_v37"
# t2c_concode_220428_v18.json

In [9]:
current_config_path = os.path.join(CONFIG_PATH, experiment_name + "_config.json")
experiment_config = json.load(open(current_config_path, "r"))

assert experiment_config['experiment_name'] == experiment_name

In [10]:
experiment_config['resume_from_checkpoint'] = False

In [11]:
assert experiment_config['resume_from_checkpoint'] == False

In [12]:
assert experiment_config['experiment_name'] == experiment_name

In [13]:
current_experiment_path = os.path.join(EXPERIMENTS_PATH, experiment_name)

In [14]:
print("current_experiment_path", current_experiment_path)

current_experiment_path /root/experiments/t2c_concode_220428_v37


In [15]:
!mkdir {current_experiment_path}

In [16]:
json.dump(experiment_config, open(current_experiment_path + \
                                  "/experiment_config.json", 
                                  "w+"
                                 )
         )

In [17]:
model, tokenizer = init_lora_model_and_tokenizer(default_model = experiment_config["default_model"],
                                                 LORA_R = experiment_config["LORA_R"],
                                                 LORA_ALPHA = experiment_config["LORA_ALPHA"],
                                                 LORA_DROPOUT = experiment_config["LORA_DROPOUT"]
                                                )



Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:

data = load_dataset("json", 
                    data_files = {"train": experiment_config["fn_train_dataset"],
                                  "eval":  experiment_config["fn_eval_dataset"]
                                 }
                   )

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-02cbc528b265cbf4/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
def verbose_model_tokenizer(model, tokenizer):
    print("tokenizer.pad_token_id", tokenizer.pad_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)
    print("tokenizer.bos_token_id", tokenizer.bos_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)

    print("model.config.pad_token_id", model.config.pad_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    print("model.config.bos_token_id", model.config.bos_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    model.print_trainable_parameters()
    
verbose_model_tokenizer(model, tokenizer)

tokenizer.pad_token_id 0
tokenizer.eos_token_id 2
tokenizer.bos_token_id 1
tokenizer.eos_token_id 2
model.config.pad_token_id 0
model.config.eos_token_id 2
model.config.bos_token_id 1
model.config.eos_token_id 2
trainable params: 8388608 || all params: 6746804224 || trainable%: 0.12433454005023165


In [20]:
# experiment_config

In [21]:
from prompter import Prompter
prompter = Prompter()

def generate_prompt(data_point):
    if "input" in data_point and data_point["input"]:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        input = data_point["input"],
                                        label = data_point["output"]
                                       )
    else:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        #input = None,
                                        label = data_point["output"]
                                       )

/root/HSE_diploma/prompter/templates/


In [22]:
experiment_config["truncation"],\
experiment_config["CUTOFF_LEN"],\
experiment_config["padding"]

# (True, 256, 'max_length')

(True, 512, 'max_length')

In [23]:
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=experiment_config["truncation"],
        max_length=experiment_config["CUTOFF_LEN"],
        padding=experiment_config["padding"]
    )
)


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=data["train"],
    eval_dataset=data['eval'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=experiment_config["MICRO_BATCH_SIZE"],
        gradient_accumulation_steps=experiment_config["GRADIENT_ACCUMULATION_STEPS"],
        warmup_steps=experiment_config["warmup_steps"],
        num_train_epochs=experiment_config["EPOCHS"],
        learning_rate=experiment_config["LEARNING_RATE"],
        fp16=experiment_config["fp16"],
        logging_steps=experiment_config["logging_steps"],        
        evaluation_strategy = experiment_config['evaluation_strategy'],
        eval_steps=experiment_config["eval_steps"],
        output_dir=current_experiment_path,#"lora-alpaca",
        save_total_limit=experiment_config["save_total_limit"],
        save_strategy = experiment_config["save_strategy"],
        
        save_steps = experiment_config["save_steps"],
        seed=experiment_config["seed"],
        logging_dir=current_experiment_path,
        logging_strategy=experiment_config["logging_strategy"],
        report_to=experiment_config["report_to"],
        load_best_model_at_end = experiment_config["load_best_model_at_end"]
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, 
                                                               mlm=experiment_config["mlm"]
                                                              ),
#     callbacks = [MyCustomCallback]
)
model.config.use_cache = experiment_config["config_use_cache"]
# print(len(trainer.optimizer.state['found_inf_per_device']))


trainer.train(resume_from_checkpoint=experiment_config["resume_from_checkpoint"])

model.save_pretrained(current_experiment_path)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.170200,1.188443
200,0.980600,1.100288
300,0.954200,1.077399
400,0.889600,1.070261
500,0.972500,1.061411
600,0.911500,1.059370
700,0.949100,1.056929
800,0.905200,1.055107
900,0.903500,1.053152
1000,0.891000,1.052750


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.179400,1.262794
200,1.115300,1.161276
300,0.991500,1.134106
400,1.003000,1.125500
500,0.930900,1.119192
600,0.972400,1.115012
700,1.053000,1.112176
800,0.965700,1.112523
900,0.982100,1.108011


generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.29.2"
}



...

{'EM': 0.0, 'BLEU': 7.699406180106954e-06, 'brevity_penalty': 0.0019735783221064072, 'ratio': 0.13835263835263836, 'translation_length': 430, 'reference_length': 3108, 'precisions_0': 0.002320185614849188, 'precisions_1': 0.0030211480362537764, 'precisions_2': 0.004329004329004329, 'precisions_3': 0.007633587786259542}


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.29.2"
}



 69%|████████████████████████████▉             | 69/100 [04:36<02:04,  4.01s/it]


KeyboardInterrupt: 

In [22]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay         118G   22G   97G  19% /


In [23]:
!ls {current_experiment_path}

1684518421.931266   events.out.tfevents.1684518421.c71a00d70d2c.1489.0
1684518421.9377556  events.out.tfevents.1684518421.c71a00d70d2c.1489.2
1684518911.7628372  events.out.tfevents.1684518911.c71a00d70d2c.1622.0
1684518911.768738   events.out.tfevents.1684518911.c71a00d70d2c.1622.2
checkpoint-500	    experiment_config.json


In [24]:
model.save_pretrained(current_experiment_path)

In [3]:
!ls /root/experiments/t2c_concode_220428_v34

1684518421.931266    checkpoint-500
1684518421.9377556   events.out.tfevents.1684518421.c71a00d70d2c.1489.0
1684518911.7628372   events.out.tfevents.1684518421.c71a00d70d2c.1489.2
1684518911.768738    events.out.tfevents.1684518911.c71a00d70d2c.1622.0
adapter_config.json  events.out.tfevents.1684518911.c71a00d70d2c.1622.2
adapter_model.bin    experiment_config.json
